In [ ]:
!pip install datasets tqdm

In [ ]:
from datasets import load_dataset
import tqdm

# 1. データセットのロード（ストリーミングモードでメモリ節約）
print("データセットに接続中...")
dataset = load_dataset("izumi-lab/wikipedia-ja-20230720", split="train", streaming=True)

# 2. 保存処理
CORPUS_FILE = "wiki_ja_subset.txt"
print(f"ダウンロードと保存を開始します: {CORPUS_FILE}")

with open(CORPUS_FILE, "w", encoding="utf-8") as f:
    # 最初の1万件だけを取得
    for i, data in enumerate(tqdm.tqdm(dataset)):
        # 改行を除去して1行にする
        text = data["text"].replace("\n", "")

        # 短すぎる記事は除外（100文字以上のみ保存）
        if len(text) > 100:
            f.write(text + "\n")

        # 10,000件でストップ
        if i >= 10000:
            break

print("✅ データセット作成完了！")


In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import ByteLevel
from tokenizers.decoders import ByteLevel as ByteLevelDecoder

# 1. モデルの初期化（BPE）
tokenizer = Tokenizer(BPE())

# 2. Pre-tokenizerの設定（バイトレベルで分割）
# GPT-2などと同様、スペースも含めて処理する設定
tokenizer.pre_tokenizer = ByteLevel(add_prefix_space=False)

# 3. Decoderの設定（IDから文字列に戻す用）
tokenizer.decoder = ByteLevelDecoder()

# 4. Trainerの設定
# vocab_size: 語彙数（日本語LLMでは32000〜64000程度が一般的）
# min_frequency: 登場回数がこれ以下のサブワードは作らない
vocab_size = 32000
trainer = BpeTrainer(
    vocab_size=vocab_size,
    min_frequency=2,
    special_tokens=["<|endoftext|>", "<|pad|>"], # 特殊トークンの定義
    show_progress=True
)

# 5. 学習実行
tokenizer.train([CORPUS_FILE], trainer)

# 6. 保存
tokenizer.save("custom_tokenizer.json")
print(f"トークナイザ学習完了。Vocab Size: {tokenizer.get_vocab_size()}")

In [ ]:
test_sentence = "生成AIの技術は日進月歩で進化しています。"
encoded = tokenizer.encode(test_sentence)

print(f"Original: {test_sentence}")
print(f"Tokens:   {encoded.tokens}")
print(f"IDs:      {encoded.ids}")

In [ ]:
# 1つずつのIDを元の文字に戻して表示する検証コード
print(f"元の文: {test_sentence}\n")
print("--- トークンごとの分割内訳 ---")

for t_id in encoded.ids:
    # IDを1つだけデコードする
    word = tokenizer.decode([t_id])
    print(f"ID: {t_id:5d} | トークン: {word}")

In [ ]:
import torch
from torch.utils.data import Dataset

class LLMPretrainDataset(Dataset):
    def __init__(self, txt_file, tokenizer, max_length=512):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.input_ids = []

        # 全テキストを読み込んでトークナイズ（メモリ注意：本番ではmmap等を使う）
        with open(txt_file, "r", encoding="utf-8") as f:
            text = f.read()

        # 一気にエンコード
        tokens = tokenizer.encode(text).ids

        # max_lengthごとに分割（簡易的なスライディングウィンドウなしの実装）
        # strideをつける場合は step = max_length (オーバーラップなし)
        for i in range(0, len(tokens) - max_length, max_length):
            self.input_ids.append(tokens[i : i + max_length])

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        # input:  x_1, x_2, ..., x_T
        # target: x_2, x_3, ..., x_{T+1} (1つ右にずらす)
        chunk = self.input_ids[idx]

        x = torch.tensor(chunk, dtype=torch.long)
        y = torch.tensor(chunk, dtype=torch.long) # 実際はずらしてLoss計算時に処理するが、ここではデータとしては同じものを返すのが通例

        return x, y

# 動作確認
dataset = LLMPretrainDataset(CORPUS_FILE, tokenizer, max_length=128)
print(f"総サンプル数: {len(dataset)}")

x, y = dataset[0]
print(f"Input shape: {x.shape}")